# **Core Features & API Development**
---
## **Step 1: Define the Models**
We need two core models:
1. **Charity**: Represents the charitable organizations.
2. **Donation**: Tracks donations and their allocations.

### **models.py**
```python
from django.db import models
from django.contrib.auth.models import User

class Charity(models.Model):
    name = models.CharField(max_length=255, unique=True)
    description = models.TextField(blank=True, null=True)
    website = models.URLField(blank=True, null=True)

    def __str__(self):
        return self.name


class Donation(models.Model):
    user = models.ForeignKey(User, on_delete=models.CASCADE, related_name="donations")
    amount = models.DecimalField(max_digits=10, decimal_places=2)
    created_at = models.DateTimeField(auto_now_add=True)
    charities = models.ManyToManyField(Charity, through="DonationAllocation")

    def __str__(self):
        return f"${self.amount} by {self.user.username}"


class DonationAllocation(models.Model):
    donation = models.ForeignKey(Donation, on_delete=models.CASCADE)
    charity = models.ForeignKey(Charity, on_delete=models.CASCADE)
    amount = models.DecimalField(max_digits=10, decimal_places=2)

    def __str__(self):
        return f"${self.amount} to {self.charity.name}"
```

---

## **Step 2: Implement the Donation Allocation Logic**
When a user makes a donation, 50% is split evenly among the selected charities, and 50% goes to the couple.

### **signals.py**
```python
from django.db.models.signals import post_save
from django.dispatch import receiver
from .models import Donation, DonationAllocation

@receiver(post_save, sender=Donation)
def allocate_donation(sender, instance, created, **kwargs):
    if created:
        charities = instance.charities.all()
        if charities.exists():
            charity_share = instance.amount * 0.5
            per_charity = charity_share / charities.count()
            
            for charity in charities:
                DonationAllocation.objects.create(
                    donation=instance,
                    charity=charity,
                    amount=per_charity
                )
```

🔹 **How It Works:**
- When a donation is saved, this function automatically calculates the allocation.
- 50% of the donation is split among selected charities.
- 50% is assumed to go to the couple (handled separately in payment processing).

---

## **Step 3: Build Django REST API for Donations**
Now, let's expose APIs for:
- Listing charities
- Making a donation
- Viewing donation history

### **serializers.py**
```python
from rest_framework import serializers
from .models import Charity, Donation, DonationAllocation

class CharitySerializer(serializers.ModelSerializer):
    class Meta:
        model = Charity
        fields = "__all__"


class DonationSerializer(serializers.ModelSerializer):
    charities = serializers.PrimaryKeyRelatedField(queryset=Charity.objects.all(), many=True)

    class Meta:
        model = Donation
        fields = ["id", "user", "amount", "charities", "created_at"]
        read_only_fields = ["user", "created_at"]

    def create(self, validated_data):
        charities = validated_data.pop("charities", [])
        donation = Donation.objects.create(**validated_data)
        donation.charities.set(charities)
        return donation


class DonationAllocationSerializer(serializers.ModelSerializer):
    charity = CharitySerializer()

    class Meta:
        model = DonationAllocation
        fields = ["charity", "amount"]
```

---

### **views.py**
```python
from rest_framework import viewsets, permissions
from rest_framework.response import Response
from .models import Charity, Donation
from .serializers import CharitySerializer, DonationSerializer

class CharityViewSet(viewsets.ModelViewSet):
    queryset = Charity.objects.all()
    serializer_class = CharitySerializer
    permission_classes = [permissions.AllowAny]  # Publicly accessible


class DonationViewSet(viewsets.ModelViewSet):
    serializer_class = DonationSerializer
    permission_classes = [permissions.IsAuthenticated]

    def get_queryset(self):
        return Donation.objects.filter(user=self.request.user)

    def perform_create(self, serializer):
        serializer.save(user=self.request.user)
```

---

### **urls.py**
```python
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from .views import CharityViewSet, DonationViewSet

router = DefaultRouter()
router.register(r"charities", CharityViewSet)
router.register(r"donations", DonationViewSet)

urlpatterns = [
    path("", include(router.urls)),
]
```

---

## **Step 4: Test API with Django REST Framework UI**
- Run the server:
  ```sh
  python manage.py runserver
  ```
- Open **http://127.0.0.1:8000/api/charities/** to view available charities.
- Open **http://127.0.0.1:8000/api/donations/** to create and track donations.

---

## **Next Steps**
✔️ Integrate Stripe/PayPal for payment processing.  
✔️ Develop frontend donation workflow (React).  